# Overview
This notebook contains my supplementary work for a bank analysis project done in Tableau.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from dateutil.relativedelta import relativedelta

# Creating Revenue-by-Month CSV from extrapolation
Ingest the loan information...

In [2]:
df = pd.read_csv('completedloan.csv')

Convert column to datetime...

In [3]:
df['fulldate'] = pd.to_datetime(df['fulldate'])

Build a function to calculate loan end date from loan start date + duration (months)...

In [4]:
# Defining the Add Months function
def add_months(start_date, delta_period):
  end_date = start_date + relativedelta(months=delta_period)
  return end_date

Apply the function to create a column of loan end dates...

In [5]:
# Apply function on the dataframe using lambda operation.
df['enddate'] = df.apply(lambda row: add_months(row['fulldate'], row['duration']), axis = 1)

Check work...

In [6]:
df

,loan_id,account_id,amount,duration,payments,status,year,month,day,fulldate,location,purpose,enddate
0,L00005657,A00003354,4980,12,415,A,2014,7,5,2014-07-05,59,car,2015-07-05
1,L00006234,A00006061,5148,12,429,C,2018,5,28,2018-05-28,1,car,2019-05-28
2,L00006699,A00008330,7656,24,319,A,2014,3,30,2014-03-30,1,car,2016-03-30
3,L00006688,A00008268,8616,24,359,C,2017,9,26,2017-09-26,1,car,2019-09-26
4,L00006312,A00006453,10944,36,304,C,2018,5,17,2018-05-17,1,car,2021-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,L00005569,A00002936,504000,60,8400,C,2018,1,20,2018-01-20,18,home,2023-01-20
678,L00005132,A00000817,538500,60,8975,C,2015,2,17,2015-02-17,64,home,2020-02-17
679,L00005447,A00002335,541200,60,9020,D,2017,11,12,2017-11-12,59,home,2022-11-12
680,L00006791,A00008926,566640,60,9444,C,2018,1,23,2018-01-23,1,home,2023-01-23


Create a list of years and months for the dataset's date range...

In [7]:
dates = pd.date_range(df['fulldate'].min(), df['fulldate'].max(), 
                      freq='MS').strftime("%Y-%m-%d").tolist()

Create initial portion of output dataframe...

In [8]:
new = df[['loan_id','account_id']]

1. Create a column for each unique month+year combination
2. Check if month+year falls during a loan's start and end date
3. Multiply True and False values (True=1, False=0) by the amount that should be paid each month
    * Result: that amount was paid that month by that account
    
Then check results...

In [9]:
for date in dates:
    new[date[:7]] = ((df.fulldate.dt.to_period('M') <= date[:7]) &
                     (df.enddate.dt.to_period('M') >= date[:7])) * df['payments']
    
new

,loan_id,account_id,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,L00005657,A00003354,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,L00006234,A00006061,0,0,0,0,0,0,0,0,...,0,0,429,429,429,429,429,429,429,429
2,L00006699,A00008330,0,0,0,0,0,0,0,319,...,0,0,0,0,0,0,0,0,0,0
3,L00006688,A00008268,0,0,0,0,0,0,0,0,...,359,359,359,359,359,359,359,359,359,359
4,L00006312,A00006453,0,0,0,0,0,0,0,0,...,0,0,304,304,304,304,304,304,304,304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,L00005569,A00002936,0,0,0,0,0,0,0,0,...,8400,8400,8400,8400,8400,8400,8400,8400,8400,8400
678,L00005132,A00000817,0,0,0,0,0,0,0,0,...,8975,8975,8975,8975,8975,8975,8975,8975,8975,8975
679,L00005447,A00002335,0,0,0,0,0,0,0,0,...,9020,9020,9020,9020,9020,9020,9020,9020,9020,9020
680,L00006791,A00008926,0,0,0,0,0,0,0,0,...,9444,9444,9444,9444,9444,9444,9444,9444,9444,9444


Output work as two options, either with the account information or just by total revenue by year+month...

In [10]:
new.to_csv('rev_by_good_account_by_date.csv')

In [11]:
new.drop(columns=['loan_id','account_id']).sum().to_csv('rev_by_day.csv')